## Julia Docker images

Available at [dockerhub](https://hub.docker.com/_/julia)

`docker pull julia:1.4.1`

In [1]:
run(pipeline(`docker images`, `grep julia`, `grep "1\.4\.1"`));

julia                                1.4.1               248f01656dfd        2 weeks ago         447MB


## Run a simple Julia Task

Let us first run a Julia task that just prints out something.

We connect to the kubernetes cluster. We then create a Pod specification using julia image. Here, along with the image name, we are also specifying the command to run (`julia`) and the arguments to it.

In [2]:
using Kuber
ctx = KuberContext();
Kuber.set_api_versions!(ctx);

┌ Info: unsupported crd.projectcalico.org/v1
└ @ Kuber /home/tan/.julia/packages/Kuber/fdQId/src/helpers.jl:131


In [3]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(time()); sleep(5); end"
                   ]
               }]
           }
       }""");

In [4]:
pod = put!(ctx, julia_pod);

We now use the `kubectl` tool to show what out julia process prints out.

In [5]:
;kubectl logs julia-pod

1.58891125299015e9
1.588911258156469e9


In [6]:
delete!(ctx, pod);

## Specifying resource allocation

Optional, but recommended. We specify the minimum amount of resource (`requests`) and the maximum (`limits`) that our process may be allowed to use if no one else is using them. Similar to `cpu` we can allocate memory resources as well.

Helps the cluster to schedule processes better.

Safeguards your processes from being throttled or terminated if they exceed available resources.

In [7]:
julia_pod = kuber_obj(ctx, """{
           "kind": "Pod",
           "metadata":{
               "name": "julia-pod",
               "namespace": "default",
               "labels": {
                   "name": "julia-pod"
               }
           },
           "spec": {
               "containers": [{
                   "name": "julia",
                   "image": "julia:1.4.1",
                   "command": ["julia"],
                   "args": [
                       "-e",
                       "while true; println(time()); sleep(5); end"
                   ],
                   "resources": {
                       "limits": {
                           "cpu": "1"
                       },
                       "requests": {
                           "cpu": "500m"
                       }
                   }
               }]
           }
       }""");

In [8]:
pod = put!(ctx, julia_pod);

In [10]:
;kubectl logs julia-pod

1.588911463589098e9
1.588911468884244e9


In [11]:
delete!(ctx, pod);